# Calculate post-TT dM/M0 and CBF time series for every ROI

Calculate the perfusion (dM/M0) time series  (i.e. vs PLD) for each ROI and filter only PLDs that are longer than the arterial transit time. Then calculate CBF using the white paper model, substituting assumed vs calculated T1 of arterial blood (aka T1a) parameter values. The best T1blood value should provide a flat response for PLD>ATT. That is because once the label reaches the voxel (PLD>TT) it just sits there and relaxes with T1b, so adjusting the CBF calculation for PLD with the correct T1a (arterial blood T1) should result in the same CBF regardless of PLD.


In [1]:
import os
import sys
import pandas as pd
import numpy as np
import subprocess
import glob
import flywheel

local_dir = '../data/subjects'

In [2]:
def refine_list(lst):
    '''
    creates a usable list of floats from the fsl output string

    :param lst: fsl output string
    :return: list of floats
    
    '''
    result_list = str(lst).split(' ')
    result_list[0] = result_list[0].replace("b'", "")
    result_list = result_list[: len(result_list) - 1] # remove extra text
    result_list = [float(e) for e in result_list]
    
    return(result_list)
    

In [3]:
cols = ['session_id', 'gray_matter','acc', 'insula', 'caudate', 'putamen']
df = pd.DataFrame(columns=cols)
for root, dirs, files in os.walk(local_dir, topdown=False):
    pw_maps = [f for f in files if 'dMM0' in f]
    if len(pw_maps) > 0:
        for pw_map in pw_maps:
            ses_name = pw_map.split('dMM0')[0][:-1] # drop the last underscore
            pw_map_path = os.path.join(root, pw_map)
            roi_dir = os.path.join(root, ses_name + "_rois")
            
            for mask in glob.glob(roi_dir+"/*roimask.nii.gz"):
                cmd1="fslmeants --transpose -i '{}' -m '{}'".format(pw_map_path, mask)
                result = refine_list(subprocess.check_output(cmd1, shell=True))
                if 'acc' in mask:
                    acc = result
                elif 'insula' in mask:
                     insula = result
                elif 'caudate' in mask:
                    caudate = result
                elif 'putamen' in mask:
                    putamen = result
                elif 'GM' in mask:
                    gray_matter = result

            
            
            df.loc[len(df.index)] = [ses_name, gray_matter, acc, insula, caudate, putamen]

df.head()

,session_id,gray_matter,acc,insula,caudate,putamen
0,sub-HC010_ses-MR_FOLLOWUP,"[0.013113, 0.018298, 0.016311, 0.015468, 0.011...","[0.014776, 0.02015, 0.015609, 0.012572, 0.0144...","[0.032526, 0.024552, 0.015869, 0.014578, 0.008...","[0.006158, 0.01231, 0.010754, 0.0105, 0.010034...","[0.010393, 0.015067, 0.012648, 0.011151, 0.008..."
1,sub-HC010_ses-MR_BASELINE,"[0.008968, 0.01323, 0.013456, 0.010283, 0.0108...","[0.003826, 0.013707, 0.01348, 0.008908, 0.0070...","[0.025325, 0.017404, 0.012282, 0.008568, 0.009...","[0.005207, 0.007674, 0.005317, 0.005354, 0.006...","[0.007632, 0.01038, 0.009006, 0.007345, 0.0067..."
2,sub-HC017_ses-MR_BASELINE,"[0.028185, 0.03348, 0.027301, 0.02228, 0.01828...","[0.031958, 0.030385, 0.022931, 0.017863, 0.015...","[0.032692, 0.028287, 0.018258, 0.016547, 0.012...","[0.018108, 0.016684, 0.012469, 0.009351, 0.008...","[0.023478, 0.021816, 0.016207, 0.011482, 0.009..."
3,sub-HC017_ses-MR_FOLLOWUP,"[0.021704, 0.022413, 0.022249, 0.01767, 0.0156...","[0.024823, 0.022764, 0.019931, 0.012293, 0.011...","[0.029969, 0.022175, 0.017119, 0.014001, 0.013...","[0.011873, 0.01338, 0.012009, 0.007465, 0.0066...","[0.018751, 0.017178, 0.014319, 0.009509, 0.008..."
4,sub-HC028_ses-MR_BASELINE,"[0.012033, 0.0174, 0.015653, 0.010333, 0.01218...","[0.01986, 0.022898, 0.015693, 0.00908, 0.01248...","[0.026282, 0.021107, 0.014825, 0.008511, 0.009...","[0.007822, 0.012201, 0.009059, 0.005354, 0.007...","[0.011194, 0.017665, 0.012325, 0.006287, 0.010..."


In [4]:
# example
list(df.loc[df['session_id']=='sub-HC001_ses-MR_BASELINE', 'acc'])[0]

[0.015712, 0.022783, 0.022847, 0.015866, 0.013916, 0.014482, 0.007363]

## Select time points based on transit time for that region
Take the time series list from the above calculation and exclude time points (i.e. PLDs) that happen before ATT (i.e. when all the blood has arrived). The PLDs in this experiment are [1, 1.22, 1.48, 1.78, 2.15,x, y]. We can determine which to keep based on how long the transit-time was using a decision tree. E.g. if the transit time was between the first (1s) and second (1.22s) delays, then we eliminate the first from the data. We do this by adding a NaN at the end, so that the resulting time series is PLDs after ATT (i.e. first PLD after, second PLD after) rather than the absolute (first (1s), second (1.22s), etc.)

In [5]:
# Check the maximum transit-time
tt_df = pd.read_csv("regionals_tts.csv")
np.max(tt_df)

session_id     sub-HC028_ses-MR_FOLLOWUP
gray_matter                  1793.280349
acc                          1885.564396
insula                       1533.890533
caudate                      1524.748148
putamen                      1624.468193
dtype: object

It looks like the maximum transit time in the dataset is not more than the fifth PLD (2.15s). So there should be at least 3 points in every time series.

In [6]:
refined_df = df.copy()

for row, row in df.iterrows(): # iterate over rows
    row_list = [row.session_id] 
    ses_id = row.session_id
    for c, value in row.items():
        if type(value) == list:
            tt = float(tt_df.loc[tt_df['session_id']==row.session_id, c])/1000 # originally in ms, but we want s
            
            # extract post-TT values
            if tt < 1:
                dms = value[0:7] # all 7
            elif 1 < tt < 1.22:
                dms = value[1:7] + [np.nan]
            elif 1.22 < tt < 1.48:
                dms = value[2:7] + [np.nan]*2
            elif 1.48 < tt < 1.78:
                dms = value[3:7] + [np.nan]*3
            elif 1.78 < tt < 2.15:
                dms = value[4:7] + [np.nan]*4 # 5,6,7 see above comment
            else:
                print("??")

            row_list.append(dms)

    refined_df.loc[len(refined_df.index)] = row_list
    
refined_df = refined_df.tail(50) # only get the newly created rows
refined_df = refined_df.reset_index(drop=True)
refined_df.head()

,session_id,gray_matter,acc,insula,caudate,putamen
0,sub-HC010_ses-MR_FOLLOWUP,"[0.015468, 0.01123, 0.010576, 0.007563, nan, n...","[0.015609, 0.012572, 0.014437, 0.010503, 0.005...","[0.015869, 0.014578, 0.008736, 0.011119, 0.007...","[0.01231, 0.010754, 0.0105, 0.010034, 0.007976...","[0.012648, 0.011151, 0.008093, 0.006949, 0.006..."
1,sub-HC010_ses-MR_BASELINE,"[0.010283, 0.010845, 0.008505, 0.007467, nan, ...","[0.01348, 0.008908, 0.007059, 0.005399, 0.0052...","[0.012282, 0.008568, 0.009383, 0.006652, 0.004...","[0.007674, 0.005317, 0.005354, 0.006507, 0.005...","[0.009006, 0.007345, 0.006773, 0.005891, 0.004..."
2,sub-HC017_ses-MR_BASELINE,"[0.027301, 0.02228, 0.018281, 0.016482, 0.0123...","[0.030385, 0.022931, 0.017863, 0.015012, 0.014...","[0.028287, 0.018258, 0.016547, 0.012406, 0.012...","[0.016684, 0.012469, 0.009351, 0.008468, 0.006...","[0.021816, 0.016207, 0.011482, 0.009983, 0.008..."
3,sub-HC017_ses-MR_FOLLOWUP,"[0.022249, 0.01767, 0.015641, 0.013513, 0.0130...","[0.022764, 0.019931, 0.012293, 0.011167, 0.012...","[0.022175, 0.017119, 0.014001, 0.013036, 0.010...","[0.01338, 0.012009, 0.007465, 0.006655, 0.0053...","[0.017178, 0.014319, 0.009509, 0.00889, 0.0074..."
4,sub-HC028_ses-MR_BASELINE,"[0.010333, 0.012185, 0.010931, 0.009333, nan, ...","[0.015693, 0.00908, 0.012488, 0.010655, 0.0076...","[0.014825, 0.008511, 0.009581, 0.007944, 0.006...","[0.009059, 0.005354, 0.007674, 0.006928, 0.005...","[0.012325, 0.006287, 0.010399, 0.007837, 0.006..."


## Pull Hematocrit data from Flywheel
The hematocrit data is essential for calculating T1a. It lives on Flywheel in each subject's custom info. 

In [11]:
fw = flywheel.Client()
project_id = '5e50277c6dea314fa72a7440'
project = fw.get(project_id)

# initialize target columns with NaNs
refined_df['Hb'] = np.nan
refined_df['T1a'] = np.nan

sessions = [fw.get_session(x.id) for x in fw.get_project_sessions(project_id)]

for ses in sessions:
    subject = fw.get(ses.parents['subject'])
    session_id = "sub-{}_ses-{}".format(subject.label, ses.label) # NOTE this is not the Flywheel definition of 'id'

    if subject.info:
        hb = subject.info['Hb']
        t1a = hb / 3 #??  conversion equation
        refined_df.loc[refined_df['session_id']==session_id, 'Hb'] = hb
        refined_df.loc[refined_df['session_id']==session_id, 'T1a'] = t1a 

refined_df.to_csv('refined_dM.csv', index=False)
refined_df.head()

,session_id,gray_matter,acc,insula,caudate,putamen,Hb,T1a
0,sub-HC010_ses-MR_FOLLOWUP,"[0.015468, 0.01123, 0.010576, 0.007563, nan, n...","[0.015609, 0.012572, 0.014437, 0.010503, 0.005...","[0.015869, 0.014578, 0.008736, 0.011119, 0.007...","[0.01231, 0.010754, 0.0105, 0.010034, 0.007976...","[0.012648, 0.011151, 0.008093, 0.006949, 0.006...",16.9,5.633333
1,sub-HC010_ses-MR_BASELINE,"[0.010283, 0.010845, 0.008505, 0.007467, nan, ...","[0.01348, 0.008908, 0.007059, 0.005399, 0.0052...","[0.012282, 0.008568, 0.009383, 0.006652, 0.004...","[0.007674, 0.005317, 0.005354, 0.006507, 0.005...","[0.009006, 0.007345, 0.006773, 0.005891, 0.004...",16.9,5.633333
2,sub-HC017_ses-MR_BASELINE,"[0.027301, 0.02228, 0.018281, 0.016482, 0.0123...","[0.030385, 0.022931, 0.017863, 0.015012, 0.014...","[0.028287, 0.018258, 0.016547, 0.012406, 0.012...","[0.016684, 0.012469, 0.009351, 0.008468, 0.006...","[0.021816, 0.016207, 0.011482, 0.009983, 0.008...",14.5,4.833333
3,sub-HC017_ses-MR_FOLLOWUP,"[0.022249, 0.01767, 0.015641, 0.013513, 0.0130...","[0.022764, 0.019931, 0.012293, 0.011167, 0.012...","[0.022175, 0.017119, 0.014001, 0.013036, 0.010...","[0.01338, 0.012009, 0.007465, 0.006655, 0.0053...","[0.017178, 0.014319, 0.009509, 0.00889, 0.0074...",14.5,4.833333
4,sub-HC028_ses-MR_BASELINE,"[0.010333, 0.012185, 0.010931, 0.009333, nan, ...","[0.015693, 0.00908, 0.012488, 0.010655, 0.0076...","[0.014825, 0.008511, 0.009581, 0.007944, 0.006...","[0.009059, 0.005354, 0.007674, 0.006928, 0.005...","[0.012325, 0.006287, 0.010399, 0.007837, 0.006...",15.5,5.166667


## Calculate CBF


In [8]:
def calc_cbf_ts(dMM0_list, T1blood):
    '''
    calculates CBF values from list of DM/M0 values

    :param dMM0_list:  the list of dM/M0 values
    :param T1blood: the assumed (1.65) or calculated T1 relaxation time of blood
    :return: list of CBF values
    
    '''
    
    plds = [1, 1.22, 1.48, 1.78, 2.15, 2.63, 3.32] # post-labeling durations
    cbf_ts = []
    
    # set the parameters
    lmbda = 0.9
    alpha = 0.85
    tau = 3.5 # labeling duration
    
    denominator = 2*alpha*T1blood*(1-np.exp(-tau/T1blood)) # only need to calculate once                         
                                   
    for i in range(len(dMM0_list)-1):
        # calculate CBF factor based on corresponding PLD value
        pld = plds[i]                                   
        numerator = 6000 * lmbda * np.exp(pld/T1blood)
        factor = numerator/denominator                           
        # multiply by deltaM over M0 to get the CBF value for this PLD                          
        cbf = dMM0_list[i] * factor
        cbf_ts.append(cbf) 
                                
    return cbf_ts
    

In [9]:
cbf_df = refined_df.copy()
for row, row in refined_df.iterrows(): #iterate over rows
    row_list = [row.session_id]
    for c, value in row.items():
        if type(value) == list:
            cbf_ts = calc_cbf_ts(value, 1.65) 
            row_list.append(cbf_ts)
    
    
    row_list.append(row.Hb)
    row_list.append(row.T1a)
#     print(row_list)
    cbf_df.loc[len(cbf_df.index)] = row_list

cbf_df = cbf_df.tail(50) # only get the newly created rows
cbf_df = cbf_df.reset_index(drop=True)
# cbf_df.to_csv('cbf_ts.csv', index=False)
cbf_df.head()

,session_id,gray_matter,acc,insula,caudate,putamen,Hb,T1a
0,sub-HC010_ses-MR_FOLLOWUP,"[62.02474901761784, 51.453691626559404, 56.727...","[62.590141415567416, 57.60247650303694, 77.436...","[63.6327089578858, 66.79358116936625, 46.85799...","[49.36156325361233, 49.27275153624397, 56.3197...","[50.71690105862622, 51.09172888047764, 43.4090...",16.9,5.633333
1,sub-HC010_ses-MR_BASELINE,"[41.23354629869176, 49.689695965274865, 45.618...","[54.05311719404503, 40.81473597590304, 37.8629...","[49.24928674905497, 39.256921625677734, 50.328...","[30.77178199904314, 24.361467353376344, 28.717...","[36.112935715843435, 33.65337177177906, 36.328...",16.9,5.633333
2,sub-HC017_ses-MR_BASELINE,"[109.47360181859223, 102.08265800888188, 98.05...","[121.84005682055326, 105.06541430887209, 95.81...","[113.42733872907652, 83.65463060709898, 88.754...","[66.90075721553762, 57.130550391056914, 50.156...","[87.47943655083725, 74.25734462971043, 61.5869...",14.5,4.833333
3,sub-HC017_ses-MR_FOLLOWUP,"[89.21571249631364, 80.9605281425917, 83.89490...","[91.28079820513656, 91.31999357158999, 65.9369...","[88.91898173426917, 78.4359525338442, 75.09830...","[53.65212967776873, 55.02291921134033, 40.0406...","[68.88163554594253, 65.60689317904756, 51.0041...",14.5,4.833333
4,sub-HC028_ses-MR_BASELINE,"[41.434040056829915, 55.82931722792755, 58.631...","[62.9269709292395, 41.60280676484055, 66.98290...","[59.446399287961235, 38.995758631669375, 51.39...","[36.32545909946987, 24.530994209136157, 41.161...","[49.421711381053775, 28.805820058430896, 55.77...",15.5,5.166667


### Refine CBF ts data based on transit time, like for dM/M0 data
Again, take the time series list from the above calculation and exclude time points (i.e. PLDs) that happen before ATT (i.e. when all the blood has arrived).

In [10]:
tt_df = pd.read_csv("regionals_tts.csv")
refined_cbf_df = cbf_df.copy()

for row, row in cbf_df.iterrows(): #iterate over rows
    row_list = [row.session_id]
    for c, value in row.items():
        if type(value) == list:
            
            tt = float(tt_df.loc[tt_df['session_id']==row.session_id, c])/1000
 
            # extract post-TT values
            if tt < 1:
                dms = value[0:7]
            elif 1 < tt < 1.22:
                dms = value[1:7] + [0]
            elif 1.22 < tt < 1.48:
                dms = value[2:7] + [0]*2
            elif 1.48 < tt < 1.78:
                dms = value[3:7] + [0]*3
            elif 1.78 < tt < 2.15:
                dms = value[4:7] + [0]*4
            else:
                print("??")

            row_list.append(dms)
            
    row_list.append(row.Hb)
    row_list.append(row.T1a)
    refined_cbf_df.loc[len(refined_cbf_df.index)] = row_list
    
refined_cbf_df = refined_cbf_df.tail(50) # only get the newly created rows
refined_cbf_df.reset_index(drop=True)
refined_cbf_df.to_csv('refined_cbf.csv', index=False)